# Project 1: Exploratory Data Analysis
##### Does socioeconomic status affect the COVID-19 cases and deaths?

Things to uncover:
* Are there higher numbers of COVID-19 cases in areas of lower income neighborhoods in the states?
* Is case morality rate higher in lower income neighborhoods in the states?
* Are individuals who are considered minorities have higher numbers of hospitalized cases / higher death rates? 

Datasets to use:
* https://apps.bea.gov/API/signup/
* http://data.ctdata.org/dataset/poverty-status-by-town

Things to consider about the data:
* Many of the deaths due to COVID-19 occurred in the senior population
* economically disadvantaged people include:
    * people living in overcrowded accommodation: this can lead to poor housing conditions, limited access to personal outdoor space and overcrowding will reduce compliance with social distancing.
    * financially poorer people are often employed in occupations that do not provide opportunities to work from home.
    * people of low SES present to healthcare services at a more advanced stage of illness, resulting in poorer health outcomes: this will likely lead to poorer health outcomes from COVID-19 for economically disadvantaged people.
    * access to health care is also determined by a person's ability to use health services ʻwith ease, and having confidence that you will be treated with respectʼ.8 This can be hindered by language barriers, patients' attitudes towards healthcare providers and the behaviour and attitudes of healthcare professionals towards minority patients

Useful links:
* COVID19 sociodemographic & socioeconmic factors research paper: https://www150.statcan.gc.ca/n1/daily-quotidien/220308/dq220308d-eng.htm#:~:text=The%20relationship%20between%20low%20income,in%20COVID%2D19%20mortality.
* Detailed income and employment statistics by state, county, and metropolitan area: https://apps.bea.gov/api/_pdf/bea_web_service_api_user_guide.pdf
* More information on factors of covid-19: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7221360/

In [1]:
# Importing dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import scipy as sp
import pprint

# Importing api key
from api_keys import BEA_api_key

### Median Income by County in the USA
Data is taken from the following sources:
+ Data containing the counties in the USA and the median income in each county and state:  https://apps.bea.gov/api/_pdf/bea_web_service_api_user_guide.pdf Page 57 (Appendix N)
+ csv file for county population in US: https://www.census.gov/data/tables/time-series/demo/popest/2020s-counties-total.html

#### Data Cleaning & Filtering

In [2]:
# Creating the median income query
MedianIncomeQuery_url = f'https://apps.bea.gov/api/data/?UserID={BEA_api_key}&method=GetData&datasetname=Regional&TableName=CAINC1&LineCode=1&Year=2020,2021&GeoFips=COUNTY&ResultFormat=json'
MedianIncomeQueryResult = requests.get(MedianIncomeQuery_url).json()

In [3]:
MedianIncomeQueryResult['BEAAPI']['Results']['Data'][0]

{'Code': 'CAINC1-1',
 'GeoFips': '01001',
 'GeoName': 'Autauga, AL',
 'TimePeriod': '2020',
 'CL_UNIT': 'Thousands of dollars',
 'UNIT_MULT': '3',
 'DataValue': '2,664,063'}

In [5]:
# Collect geofips, geoname, time period (only between 2020-2022), CL_UNIT, and DataValue

medianIncome = [] # Creating an empty array for storing the data

for i in range(len(MedianIncomeQueryResult['BEAAPI']['Results']['Data'])):
    medianIncome.append({
    'GeoName' : MedianIncomeQueryResult['BEAAPI']['Results']['Data'][i]['GeoName'],
    'TimePeriod' : MedianIncomeQueryResult['BEAAPI']['Results']['Data'][i]['TimePeriod'],
    'CL_UNIT' : MedianIncomeQueryResult['BEAAPI']['Results']['Data'][i]['CL_UNIT'],
    'DataValue' : MedianIncomeQueryResult['BEAAPI']['Results']['Data'][i]['DataValue']
    })


In [6]:
# Creating a dataframe
medianIncome_df = (pd.DataFrame(medianIncome)).rename(columns = {'GeoName' : 'County, State'})

In [8]:
medianIncome_df.to_csv('medianIncome.csv')

In [41]:
medianIncome_df['County, State'] = medianIncome_df['County, State'].map(lambda x: x.rstrip('(Independent City)').rstrip('*'))

In [42]:
medianIncome_df

,"County, State",TimePeriod,CL_UNIT,DataValue
0,"Autauga, AL",2020,Thousands of dollars,"2,664,063"
1,"Autauga, AL",2021,Thousands of dollars,"2,857,062"
2,"Baldwin, AL",2020,Thousands of dollars,"11,971,269"
3,"Baldwin, AL",2021,Thousands of dollars,"13,079,505"
4,"Barbour, AL",2020,Thousands of dollars,"934,683"
...,...,...,...,...
6275,"Uinta, WY",2021,Thousands of dollars,"911,174"
6276,"Washakie, WY",2020,Thousands of dollars,"438,250"
6277,"Washakie, WY",2021,Thousands of dollars,"464,107"
6278,"Weston, WY",2020,Thousands of dollars,"323,424"


In [43]:
medianIncome_df.to_csv('medianIncome.csv')

#### Country Population DataFrame
Using csv read in pandas to read the data about the population size from years 2020-2022 for each County in each State in the United States.

In [44]:
countyPops = ((pd.read_csv('co-est2022-pop.csv')).dropna()).rename(columns = {'table with row headers in column A and column headers in rows 3 through 4 (leading dots indicate sub-parts)' : 'Location',
                                                                             'Unnamed: 2' : 'Population in 2020',
                                                                             'Unnamed: 3' : 'Population in 2021',
                                                                             'Unnamed: 4' : 'Population in 2022'})
countyPops = (countyPops.drop(['Unnamed: 1'], axis = 1).reset_index()).drop(['index'], axis = 1)

In [45]:
# Viewing the DataFrame
countyPops

,Location,Population in 2020,Population in 2021,Population in 2022
0,United States,"331,511,512","332,031,554","333,287,557"
1,".Autauga County, Alabama","58,902","59,210","59,759"
2,".Baldwin County, Alabama","233,219","239,361","246,435"
3,".Barbour County, Alabama","24,960","24,539","24,706"
4,".Bibb County, Alabama","22,183","22,370","22,005"
...,...,...,...,...
3140,".Sweetwater County, Wyoming","42,190","41,582","41,345"
3141,".Teton County, Wyoming","23,377","23,622","23,287"
3142,".Uinta County, Wyoming","20,457","20,655","20,712"
3143,".Washakie County, Wyoming","7,658","7,712","7,719"


In [46]:
splitting_location = (countyPops['Location'].str.split(", ", expand = True)).rename(columns = {0 : 'County',
                                                                                             1: 'State'})
splitting_location['County'] = splitting_location['County'].map(lambda x: x.lstrip('.').rstrip('ci').rstrip('County'))

In [47]:
splitting_location.to_csv('pops.csv')

In [48]:
splitting_location

,County,State
0,United States,None
1,Autauga,Alabama
2,Baldwin,Alabama
3,Barbour,Alabama
4,Bibb,Alabama
...,...,...
3140,Sweetwater,Wyoming
3141,Teton,Wyoming
3142,Uinta,Wyoming
3143,Washakie,Wyoming


In [49]:
# Taken from: https://gist.github.com/rogerallen/1583593
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [50]:
county_state_df = splitting_location.replace({'State': us_state_to_abbrev})
county_state_df['County'] = splitting_location['County'].map(lambda x: x.rstrip(' '))

In [51]:
county_state_df

,County,State
0,United States,None
1,Autauga,AL
2,Baldwin,AL
3,Barbour,AL
4,Bibb,AL
...,...,...
3140,Sweetwater,WY
3141,Teton,WY
3142,Uinta,WY
3143,Washakie,WY


In [52]:
# Combining the two columns using DataFrame.apply() and lambda function
county_state_df['County, State'] = county_state_df['County'].astype(str) + ', ' + county_state_df['State']
county_state_df = county_state_df.drop(['County', 'State'], axis = 1)

In [53]:
county_state_df

,"County, State"
0,NaN
1,"Autauga, AL"
2,"Baldwin, AL"
3,"Barbour, AL"
4,"Bibb, AL"
...,...
3140,"Sweetwater, WY"
3141,"Teton, WY"
3142,"Uinta, WY"
3143,"Washakie, WY"


In [54]:
countyPops_df = (countyPops.join(county_state_df).drop(['Location'], axis = 1)).tail(-1)

In [55]:
countyPops_df

,Population in 2020,Population in 2021,Population in 2022,"County, State"
1,"58,902","59,210","59,759","Autauga, AL"
2,"233,219","239,361","246,435","Baldwin, AL"
3,"24,960","24,539","24,706","Barbour, AL"
4,"22,183","22,370","22,005","Bibb, AL"
5,"59,102","59,085","59,512","Blount, AL"
...,...,...,...,...
3140,"42,190","41,582","41,345","Sweetwater, WY"
3141,"23,377","23,622","23,287","Teton, WY"
3142,"20,457","20,655","20,712","Uinta, WY"
3143,"7,658","7,712","7,719","Washakie, WY"


In [56]:
# Checking for duplicate entries
duplicate_counties = countyPops_df.loc[countyPops_df.duplicated(subset = ['County, State']), 'County, State'].unique()
duplicate_counties

array([], dtype=object)

#### Merging DataFrames

In [57]:
# The OUTER JOIN is used to retrieve all records from tables
# There are some NULL values in the table with missing information
merged_df = medianIncome_df.merge(countyPops_df, how='outer', on='County, State')
merged_df

,"County, State",TimePeriod,CL_UNIT,DataValue,Population in 2020,Population in 2021,Population in 2022
0,"Autauga, AL",2020,Thousands of dollars,"2,664,063","58,902","59,210","59,759"
1,"Autauga, AL",2021,Thousands of dollars,"2,857,062","58,902","59,210","59,759"
2,"Baldwin, AL",2020,Thousands of dollars,"11,971,269","233,219","239,361","246,435"
3,"Baldwin, AL",2021,Thousands of dollars,"13,079,505","233,219","239,361","246,435"
4,"Barbour, AL",2020,Thousands of dollars,"934,683","24,960","24,539","24,706"
...,...,...,...,...,...,...,...
6726,"Waukesha, WI",NaN,NaN,NaN,"407,467","409,080","410,434"
6727,"Waupaca, WI",NaN,NaN,NaN,"51,791","51,992","51,488"
6728,"Waushara, WI",NaN,NaN,NaN,"24,549","24,797","24,999"
6729,"Winnebago, WI",NaN,NaN,NaN,"171,800","170,554","170,718"


In [58]:
# Saving the merged DataFrame to excel
merged_df.to_csv('county_pops_and_income.csv')

In [59]:
# Dropping the NULL values
cleaned_df = merged_df.dropna()
cleaned_df

,"County, State",TimePeriod,CL_UNIT,DataValue,Population in 2020,Population in 2021,Population in 2022
0,"Autauga, AL",2020,Thousands of dollars,"2,664,063","58,902","59,210","59,759"
1,"Autauga, AL",2021,Thousands of dollars,"2,857,062","58,902","59,210","59,759"
2,"Baldwin, AL",2020,Thousands of dollars,"11,971,269","233,219","239,361","246,435"
3,"Baldwin, AL",2021,Thousands of dollars,"13,079,505","233,219","239,361","246,435"
4,"Barbour, AL",2020,Thousands of dollars,"934,683","24,960","24,539","24,706"
...,...,...,...,...,...,...,...
6275,"Uinta, WY",2021,Thousands of dollars,"911,174","20,457","20,655","20,712"
6276,"Washakie, WY",2020,Thousands of dollars,"438,250","7,658","7,712","7,719"
6277,"Washakie, WY",2021,Thousands of dollars,"464,107","7,658","7,712","7,719"
6278,"Weston, WY",2020,Thousands of dollars,"323,424","6,818","6,766","6,860"


In [60]:
cleaned_df['County, State'].value_counts()

Autauga, AL      2
Fayette, OH      2
Coshocton, OH    2
Crawford, OH     2
Cuyahoga, OH     2
                ..
Ford, KS         2
Franklin, KS     2
Geary, KS        2
Gove, KS         2
Weston, WY       2
Name: County, State, Length: 2693, dtype: int64

#### Data Processing

In [61]:
# Required for turning DataValue column and population columns into integers
import locale
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 

'en_US.UTF-8'

In [63]:
for index, row in cleaned_df.iterrows():
    row['DataValue'] = locale.atoi(row['DataValue']) * 1000
    row['Population in 2020'] = locale.atoi(row['Population in 2020'])
    row['Population in 2021'] = locale.atoi(row['Population in 2021'])
    row['Population in 2022'] = locale.atoi(row['Population in 2022'])

In [64]:
cleaned_df

,"County, State",TimePeriod,CL_UNIT,DataValue,Population in 2020,Population in 2021,Population in 2022
0,"Autauga, AL",2020,Thousands of dollars,2664063000,58902,59210,59759
1,"Autauga, AL",2021,Thousands of dollars,2857062000,58902,59210,59759
2,"Baldwin, AL",2020,Thousands of dollars,11971269000,233219,239361,246435
3,"Baldwin, AL",2021,Thousands of dollars,13079505000,233219,239361,246435
4,"Barbour, AL",2020,Thousands of dollars,934683000,24960,24539,24706
...,...,...,...,...,...,...,...
6275,"Uinta, WY",2021,Thousands of dollars,911174000,20457,20655,20712
6276,"Washakie, WY",2020,Thousands of dollars,438250000,7658,7712,7719
6277,"Washakie, WY",2021,Thousands of dollars,464107000,7658,7712,7719
6278,"Weston, WY",2020,Thousands of dollars,323424000,6818,6766,6860


In [75]:
cleaned_df.to_csv('cleaned_df.csv')

In [76]:
print(cleaned_df[cleaned_df['Population in 2020'].isnull()])


Empty DataFrame
Columns: [County, State, TimePeriod, CL_UNIT, DataValue, Population in 2020, Population in 2021, Population in 2022]
Index: []


In [73]:
for index, row in merged_df.iterrows():
    if row['TimePeriod'] == '2020':
        row['Average Median Income'] = row['DataValue'] / int(row['Population in 2020'])
    else:
        row['Average Median Income'] = row['DataValue'] / int(row['Population in 2021'])

ValueError: cannot convert float NaN to integer

In [ ]:
if merged_df['TimePeriod'] == merged_df[]
merged_df['Average Median Income'] = merged_df['DataValue'] / merged_df

In [ ]:
max_medianIncome = max(merged_df['DataValue'])
min_medianIncome = min(merged_df['DataValue'])

print(max_medianIncome, min_medianIncome)

In [ ]:
# Poverty scale is measured from 1-10. 1 being poverty and 10 being rich.
# Creating the bins in which the poverty data will be held
bins = [min_medianIncome, ]

# Creating the poverty scale for the different bins